Contains the R models used to analyze the number of review comments before and after the introduction of Travis CI. Looks at whether a boolean variable `IsAfterTravisIntroduction` can be used to predict the number of review comments under a pull request. 

In [141]:
filename <- 'generated/num_of_review_comments.csv'

NumOfReviewCommentsData <- read.csv(file=filename, header=TRUE, sep=",")

summary(NumOfReviewCommentsData)

 ReviewComments    ShareReviewComments GeneralComments   
 Min.   :  0.000   Min.   :  0.00      Min.   :   0.000  
 1st Qu.:  0.000   1st Qu.:  0.00      1st Qu.:   0.000  
 Median :  0.000   Median :  0.00      Median :   1.000  
 Mean   :  1.413   Mean   : 11.34      Mean   :   2.885  
 3rd Qu.:  0.000   3rd Qu.:  0.00      3rd Qu.:   3.000  
 Max.   :404.000   Max.   :100.00      Max.   :1035.000  
                                                         
 GeneralCommentsDiscussingBuild   Additions         Deletions        
 Min.   :0                      Min.   :      0   Min.   :      0.0  
 1st Qu.:0                      1st Qu.:      1   1st Qu.:      0.0  
 Median :0                      Median :      9   Median :      2.0  
 Mean   :0                      Mean   :   1822   Mean   :    513.3  
 3rd Qu.:0                      3rd Qu.:     59   3rd Qu.:     17.0  
 Max.   :0                      Max.   :6858896   Max.   :1186576.0  
                                              

In [142]:
library(lmerTest)
library(MuMIn)
library(VIF)
library(sqldf)

vif.mer <- function (fit) {
    ## adapted from rms::vif
    
    v <- vcov(fit)
    nam <- names(fixef(fit))

    ## exclude intercepts
    ns <- sum(1 * (nam == "Intercept" | nam == "(Intercept)"))
    if (ns > 0) {
        v <- v[-(1:ns), -(1:ns), drop = FALSE]
        nam <- nam[-(1:ns)]
    }
    
    d <- diag(v)^0.5
    v <- diag(solve(v/(d %o% d)))
    names(v) <- nam
    v
}

In [143]:
hasReviewComments <- sqldf("select *
                      from 'NumOfReviewCommentsData' 
                      where ReviewComments > 0")

hasReviewCommentsAndFromOutsider <- sqldf("select *
                      from 'NumOfReviewCommentsData' 
                      where ReviewComments > 0 and FromOutsider = 'True'
                        and ProjectName in (select ProjectName from 'NumOfReviewCommentsData'
                                            where ReviewComments > 0 and FromOutsider = 'True'
                                            GROUP BY ProjectName
                                            HAVING Count() > 1)")

# Clear unused factors
hasReviewCommentsAndFromOutsider$ProjectName <- factor(hasReviewCommentsAndFromOutsider$ProjectName)
hasReviewCommentsAndFromOutsider$ProjectLanguage <- factor(hasReviewCommentsAndFromOutsider$ProjectLanguage)
hasReviewCommentsAndFromOutsider$IsMerged <- factor(hasReviewCommentsAndFromOutsider$IsMerged)

hasGeneralComments <- sqldf("select *
                      from 'NumOfReviewCommentsData' 
                      where GeneralComments > 0")

hasGeneralCommentsAndFromOutsider <- sqldf("select *
                      from 'NumOfReviewCommentsData' 
                      where GeneralComments > 0 and FromOutsider = 'True'
                        and ProjectName in (select ProjectName from 'NumOfReviewCommentsData'
                                            where GeneralComments > 0 and FromOutsider = 'True'
                                            GROUP BY ProjectName
                                            HAVING Count() > 1)")

# Clear unused factors
hasGeneralCommentsAndFromOutsider$ProjectName <- factor(hasGeneralCommentsAndFromOutsider$ProjectName)
hasGeneralCommentsAndFromOutsider$ProjectLanguage <- factor(hasGeneralCommentsAndFromOutsider$ProjectLanguage)
hasGeneralCommentsAndFromOutsider$IsMerged <- factor(hasGeneralCommentsAndFromOutsider$IsMerged)

summary(hasGeneralCommentsAndFromOutsider)

 ReviewComments    ShareReviewComments GeneralComments  
 Min.   :  0.000   Min.   : 0.00       Min.   :   1.00  
 1st Qu.:  0.000   1st Qu.: 0.00       1st Qu.:   1.00  
 Median :  0.000   Median : 0.00       Median :   2.00  
 Mean   :  1.773   Mean   :11.18       Mean   :   4.48  
 3rd Qu.:  0.000   3rd Qu.: 0.00       3rd Qu.:   5.00  
 Max.   :352.000   Max.   :99.72       Max.   :1035.00  
                                                        
 GeneralCommentsDiscussingBuild   Additions         Deletions        
 Min.   :0                      Min.   :      0   Min.   :      0.0  
 1st Qu.:0                      1st Qu.:      0   1st Qu.:      0.0  
 Median :0                      Median :      5   Median :      1.0  
 Mean   :0                      Mean   :   1100   Mean   :    358.1  
 3rd Qu.:0                      3rd Qu.:     44   3rd Qu.:      9.0  
 Max.   :0                      Max.   :6858896   Max.   :1186576.0  
                                                      

# Review comments

In [144]:
modelNumberReviewComments = lmer(log(ReviewComments) ~ 
            log(Additions + 1) +
            log(Deletions + 1) +
            IsMerged +
            log(Commits + 1) +
            log(Assignees + 1) + 
            log(ChangedFiles + 1) + 
            log(NumOfUniqueUsers + 1) +    
            log(PRsOpened + 1) +
            log(TotalBuilds + 1) +
            NewContributor + 
            #log(PrOpenedDaysAfterProjectStart + 1) +
            FromOutsider + 
            IsAfter + 
            (1|ProjectLanguage) +
            (1|ProjectName),
          data= hasReviewComments,
          REML=FALSE)
summary(modelNumberReviewComments)
r.squaredGLMM(modelNumberReviewComments)
vif.mer(modelNumberReviewComments)
anova(modelNumberReviewComments)

print("---------------------------")

modelNumberReviewCommentsOutsider = lmer(log(ReviewComments) ~ 
            log(Additions + 1) +
            log(Deletions + 1) +
            #IsMerged +
            log(Commits + 1) +
            log(Assignees + 1) + 
            log(ChangedFiles + 1) + 
            log(NumOfUniqueUsers + 1) +    
            log(PRsOpened + 1) +
            log(TotalBuilds + 1) +
            NewContributor + 
            #log(PrOpenedDaysAfterProjectStart + 1) +
            #FromOutsider + 
            IsAfter +
            (1|ProjectLanguage) +
            (1|ProjectName),
          data= hasReviewCommentsAndFromOutsider,
          REML=FALSE)
summary(modelNumberReviewCommentsOutsider)
r.squaredGLMM(modelNumberReviewCommentsOutsider)
vif.mer(modelNumberReviewCommentsOutsider)
anova(modelNumberReviewCommentsOutsider)


Correlation matrix not shown by default, as p = 13 > 12.
Use print(obj, correlation=TRUE)  or
	 vcov(obj)	 if you need it



Linear mixed model fit by maximum likelihood t-tests use Satterthwaite
  approximations to degrees of freedom [lmerMod]
Formula: log(ReviewComments) ~ log(Additions + 1) + log(Deletions + 1) +  
    IsMerged + log(Commits + 1) + log(Assignees + 1) + log(ChangedFiles +  
    1) + log(NumOfUniqueUsers + 1) + log(PRsOpened + 1) + log(TotalBuilds +  
    1) + NewContributor + FromOutsider + IsAfter + (1 | ProjectLanguage) +  
    (1 | ProjectName)
   Data: hasReviewComments

     AIC      BIC   logLik deviance df.resid 
 96872.9  97009.9 -48420.4  96840.9    38543 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.8792 -0.6785 -0.0335  0.6549  5.2060 

Random effects:
 Groups          Name        Variance Std.Dev.
 ProjectName     (Intercept) 0.05275  0.2297  
 ProjectLanguage (Intercept) 0.00000  0.0000  
 Residual                    0.71483  0.8455  
Number of obs: 38559, groups:  ProjectName, 154; ProjectLanguage, 31

Fixed effects:
                            Estimate Std

R2m       R2c 
0.2876274 0.3365795

log(Additions + 1)        log(Deletions + 1)              IsMergedTrue 
                 2.963352                  2.346129                  1.054351 
         log(Commits + 1)        log(Assignees + 1)     log(ChangedFiles + 1) 
                 1.603189                  1.008765                  3.344705 
log(NumOfUniqueUsers + 1)        log(PRsOpened + 1)      log(TotalBuilds + 1) 
                 1.081677                  2.367500                  2.734133 
       NewContributorTrue          FromOutsiderTrue               IsAfterTrue 
                 1.481871                  1.666052                  2.672179

,Sum Sq,Mean Sq,NumDF,DenDF,F.value,Pr(>F)
log(Additions + 1),728.7135933,728.7135933,1,38552.44,1019.428552,0.000000e+00
log(Deletions + 1),0.9761829,0.9761829,1,38544.50,1.365624,2.425724e-01
IsMerged,30.2885538,30.2885538,1,38558.46,42.371951,7.638912e-11
log(Commits + 1),756.2787728,756.2787728,1,38520.37,1057.990658,0.000000e+00
log(Assignees + 1),65.9257668,65.9257668,1,35824.14,92.226369,0.000000e+00
log(ChangedFiles + 1),39.1300957,39.1300957,1,38557.38,54.740761,1.403322e-13
log(NumOfUniqueUsers + 1),4266.6986138,4266.6986138,1,37760.67,5968.866822,0.000000e+00
log(PRsOpened + 1),6.7388994,6.7388994,1,38075.69,9.427334,2.139253e-03
log(TotalBuilds + 1),4.9499115,4.9499115,1,38495.15,6.924643,8.504987e-03
NewContributor,16.6447783,16.6447783,1,38558.24,23.285091,1.402119e-06


[1] "---------------------------"


Linear mixed model fit by maximum likelihood t-tests use Satterthwaite
  approximations to degrees of freedom [lmerMod]
Formula: log(ReviewComments) ~ log(Additions + 1) + log(Deletions + 1) +  
    log(Commits + 1) + log(Assignees + 1) + log(ChangedFiles +  
    1) + log(NumOfUniqueUsers + 1) + log(PRsOpened + 1) + log(TotalBuilds +  
    1) + NewContributor + IsAfter + (1 | ProjectLanguage) + (1 |  
    ProjectName)
   Data: hasReviewCommentsAndFromOutsider

     AIC      BIC   logLik deviance df.resid 
 52748.5  52859.7 -26360.2  52720.5    20822 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.7222 -0.6806 -0.0339  0.6553  5.2986 

Random effects:
 Groups          Name        Variance Std.Dev.
 ProjectName     (Intercept) 0.05611  0.2369  
 ProjectLanguage (Intercept) 0.00000  0.0000  
 Residual                    0.72483  0.8514  
Number of obs: 20836, groups:  ProjectName, 154; ProjectLanguage, 31

Fixed effects:
                            Estimate Std. Error    

R2m       R2c 
0.2844319 0.3358447

log(Additions + 1)        log(Deletions + 1)          log(Commits + 1) 
                 3.183395                  2.270375                  1.586844 
       log(Assignees + 1)     log(ChangedFiles + 1) log(NumOfUniqueUsers + 1) 
                 1.011605                  3.507733                  1.072043 
       log(PRsOpened + 1)      log(TotalBuilds + 1)        NewContributorTrue 
                 1.658355                  2.939657                  1.616208 
              IsAfterTrue 
                 2.692193

,Sum Sq,Mean Sq,NumDF,DenDF,F.value,Pr(>F)
log(Additions + 1),303.4699799,303.4699799,1,20823.21,418.6765738,0.000000e+00
log(Deletions + 1),0.6747805,0.6747805,1,20826.27,0.9309481,3.346279e-01
log(Commits + 1),499.9500777,499.9500777,1,20789.94,689.7465958,0.000000e+00
log(Assignees + 1),22.1603585,22.1603585,1,18833.83,30.5731163,3.257885e-08
log(ChangedFiles + 1),30.1478611,30.1478611,1,20833.18,41.5929220,1.148674e-10
log(NumOfUniqueUsers + 1),2420.6336393,2420.6336393,1,20227.96,3339.5810643,0.000000e+00
log(PRsOpened + 1),2.2161892,2.2161892,1,20536.11,3.0575231,8.037903e-02
log(TotalBuilds + 1),10.7448746,10.7448746,1,20776.36,14.8239615,1.183800e-04
NewContributor,5.2173184,5.2173184,1,20826.78,7.1979739,7.304356e-03
IsAfter,0.7429572,0.7429572,1,20789.92,1.0250068,3.113461e-01


# General comments

In [148]:
modelNumberGeneralComments = lmer(log(GeneralComments) ~ 
            log(Additions + 1) +
            log(Deletions + 1) +
            IsMerged +
            log(Commits + 1) +
            log(Assignees + 1) + 
            log(ChangedFiles + 1) + 
            log(NumOfUniqueUsers + 1) +    
            log(PRsOpened + 1) +
            log(TotalBuilds + 1) +
            NewContributor + 
            FromOutsider + 
            #log(PrOpenedDaysAfterProjectStart + 1) +
            IsAfter + 
            (1|ProjectLanguage) +
            (1|ProjectName),
          data= hasGeneralComments, 
          REML=FALSE)
summary(modelNumberGeneralComments)
r.squaredGLMM(modelNumberGeneralComments)
vif.mer(modelNumberGeneralComments)
anova(modelNumberGeneralComments)

print("---------------------------")

modelNumberGeneralCommentsOutsider = lmer(log(GeneralComments) ~ 
            log(Additions + 1) +
            log(Deletions + 1) +
            #IsMerged +
            log(Commits + 1) +
            log(Assignees + 1) + 
            log(ChangedFiles + 1) + 
            log(NumOfUniqueUsers + 1) +    
            log(PRsOpened + 1) +
            log(TotalBuilds + 1) +
            NewContributor + 
            #log(PrOpenedDaysAfterProjectStart + 1) +
            #FromOutsider + 
            IsAfter +
            (1|ProjectLanguage) +
            (1|ProjectName),
          data= hasGeneralCommentsAndFromOutsider,
          REML=FALSE)
summary(modelNumberGeneralCommentsOutsider)
r.squaredGLMM(modelNumberGeneralCommentsOutsider)
vif.mer(modelNumberGeneralCommentsOutsider)
anova(modelNumberGeneralCommentsOutsider)


Correlation matrix not shown by default, as p = 13 > 12.
Use print(obj, correlation=TRUE)  or
	 vcov(obj)	 if you need it



Linear mixed model fit by maximum likelihood t-tests use Satterthwaite
  approximations to degrees of freedom [lmerMod]
Formula: log(GeneralComments) ~ log(Additions + 1) + log(Deletions + 1) +  
    IsMerged + log(Commits + 1) + log(Assignees + 1) + log(ChangedFiles +  
    1) + log(NumOfUniqueUsers + 1) + log(PRsOpened + 1) + log(TotalBuilds +  
    1) + NewContributor + FromOutsider + IsAfter + (1 | ProjectLanguage) +  
    (1 | ProjectName)
   Data: hasGeneralComments

     AIC      BIC   logLik deviance df.resid 
176692.7 176849.0 -88330.3 176660.7   129383 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-5.8207 -0.4821 -0.1061  0.4273  8.7410 

Random effects:
 Groups          Name        Variance  Std.Dev. 
 ProjectName     (Intercept) 2.069e-02 1.438e-01
 ProjectLanguage (Intercept) 1.146e-14 1.071e-07
 Residual                    2.283e-01 4.778e-01
Number of obs: 129399, groups:  ProjectName, 154; ProjectLanguage, 31

Fixed effects:
                            E

R2m       R2c 
0.6924689 0.7180250

log(Additions + 1)        log(Deletions + 1)              IsMergedTrue 
                 3.068897                  2.514761                  1.061534 
         log(Commits + 1)        log(Assignees + 1)     log(ChangedFiles + 1) 
                 1.652496                  1.012706                  3.399903 
log(NumOfUniqueUsers + 1)        log(PRsOpened + 1)      log(TotalBuilds + 1) 
                 1.093522                  2.438123                  2.724197 
       NewContributorTrue          FromOutsiderTrue               IsAfterTrue 
                 1.540902                  1.644199                  2.613683

,Sum Sq,Mean Sq,NumDF,DenDF,F.value,Pr(>F)
log(Additions + 1),8.808220e+00,8.808220e+00,1,129373.7,3.858899e+01,5.247160e-10
log(Deletions + 1),3.087480e+00,3.087480e+00,1,129331.3,1.352631e+01,2.353368e-04
IsMerged,4.038402e+01,4.038402e+01,1,129398.8,1.769232e+02,0.000000e+00
log(Commits + 1),2.254490e+02,2.254490e+02,1,129387.3,9.876965e+02,0.000000e+00
log(Assignees + 1),6.558741e+01,6.558741e+01,1,127108.6,2.873398e+02,0.000000e+00
log(ChangedFiles + 1),1.399111e-02,1.399111e-02,1,129362.7,6.129533e-02,8.044606e-01
log(NumOfUniqueUsers + 1),5.333520e+04,5.333520e+04,1,129351.5,2.336626e+05,0.000000e+00
log(PRsOpened + 1),2.120558e+00,2.120558e+00,1,129143.9,9.290208e+00,2.304285e-03
log(TotalBuilds + 1),8.770465e+00,8.770465e+00,1,129379.4,3.842359e+01,5.711107e-10
NewContributor,1.485660e-01,1.485660e-01,1,129386.0,6.508709e-01,4.198030e-01


[1] "---------------------------"


Linear mixed model fit by maximum likelihood t-tests use Satterthwaite
  approximations to degrees of freedom [lmerMod]
Formula: log(GeneralComments) ~ log(Additions + 1) + log(Deletions + 1) +  
    log(Commits + 1) + log(Assignees + 1) + log(ChangedFiles +  
    1) + log(NumOfUniqueUsers + 1) + log(PRsOpened + 1) + log(TotalBuilds +  
    1) + NewContributor + IsAfter + (1 | ProjectLanguage) + (1 |  
    ProjectName)
   Data: hasGeneralCommentsAndFromOutsider

     AIC      BIC   logLik deviance df.resid 
103956.0 104085.6 -51964.0 103928.0    77772 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-6.1467 -0.4864 -0.1450  0.4321  8.9977 

Random effects:
 Groups          Name        Variance Std.Dev.
 ProjectName     (Intercept) 0.0170   0.1304  
 ProjectLanguage (Intercept) 0.0000   0.0000  
 Residual                    0.2213   0.4704  
Number of obs: 77786, groups:  ProjectName, 154; ProjectLanguage, 31

Fixed effects:
                            Estimate Std. Error  

R2m       R2c 
0.7229730 0.7427321

log(Additions + 1)        log(Deletions + 1)          log(Commits + 1) 
                 3.264992                  2.449293                  1.582517 
       log(Assignees + 1)     log(ChangedFiles + 1) log(NumOfUniqueUsers + 1) 
                 1.014669                  3.531502                  1.080465 
       log(PRsOpened + 1)      log(TotalBuilds + 1)        NewContributorTrue 
                 1.741366                  2.838692                  1.692114 
              IsAfterTrue 
                 2.624619

,Sum Sq,Mean Sq,NumDF,DenDF,F.value,Pr(>F)
log(Additions + 1),1.406754e+01,1.406754e+01,1,77780.07,6.356797e+01,1.554312e-15
log(Deletions + 1),6.712771e-01,6.712771e-01,1,77748.89,3.033347e+00,8.157355e-02
log(Commits + 1),1.356833e+02,1.356833e+02,1,77785.57,6.131215e+02,0.000000e+00
log(Assignees + 1),3.457220e+01,3.457220e+01,1,75032.94,1.562238e+02,0.000000e+00
log(ChangedFiles + 1),9.801081e-01,9.801081e-01,1,77772.10,4.428884e+00,3.533882e-02
log(NumOfUniqueUsers + 1),3.649012e+04,3.649012e+04,1,77711.61,1.648905e+05,0.000000e+00
log(PRsOpened + 1),2.823659e+00,2.823659e+00,1,77483.72,1.275947e+01,3.544262e-04
log(TotalBuilds + 1),1.512022e+00,1.512022e+00,1,77780.15,6.832479e+00,8.953221e-03
NewContributor,1.706590e-01,1.706590e-01,1,77759.11,7.711689e-01,3.798584e-01
IsAfter,1.201893e+01,1.201893e+01,1,77780.68,5.431077e+01,1.727507e-13


In [149]:
modelShareReviewComments = lmer(log(ShareReviewComments + 1) ~ 
            log(Additions + 1) +
            log(Deletions + 1) +
            IsMerged +
            log(Commits + 1) +
            log(Assignees + 1) + 
            log(ChangedFiles + 1) +             
            log(NumOfUniqueUsers + 1) +    
            log(PRsOpened + 1) +
            log(TotalBuilds + 1) +
            NewContributor + 
            FromOutsider +
            IsAfter + 
            (1|ProjectLanguage) +
            (1|ProjectName),
          data= NumOfReviewCommentsData, 
          REML=FALSE)
summary(modelShareReviewComments)
r.squaredGLMM(modelShareReviewComments)
vif.mer(modelShareReviewComments)
anova(modelShareReviewComments)

print("---------------------------")

modelShareReviewCommentsOutsider = lmer(log(ShareReviewComments + 1) ~ 
            log(Additions + 1) +
            log(Deletions + 1) +
            #IsMerged +
            log(Commits + 1) +
            log(Assignees + 1) + 
            log(ChangedFiles + 1) + 
            log(NumOfUniqueUsers + 1) +    
            log(PRsOpened + 1) +
            log(TotalBuilds + 1) +
            NewContributor + 
            #log(PrOpenedDaysAfterProjectStart + 1) +
            #FromOutsider + 
            IsAfter +
            (1|ProjectLanguage) +
            (1|ProjectName),
          data= hasReviewCommentsAndFromOutsider,
          REML=FALSE)
summary(modelShareReviewCommentsOutsider)
r.squaredGLMM(modelShareReviewCommentsOutsider)
vif.mer(modelShareReviewCommentsOutsider)
anova(modelShareReviewCommentsOutsider)


Correlation matrix not shown by default, as p = 13 > 12.
Use print(obj, correlation=TRUE)  or
	 vcov(obj)	 if you need it



Linear mixed model fit by maximum likelihood t-tests use Satterthwaite
  approximations to degrees of freedom [lmerMod]
Formula: log(ShareReviewComments + 1) ~ log(Additions + 1) + log(Deletions +  
    1) + IsMerged + log(Commits + 1) + log(Assignees + 1) + log(ChangedFiles +  
    1) + log(NumOfUniqueUsers + 1) + log(PRsOpened + 1) + log(TotalBuilds +  
    1) + NewContributor + FromOutsider + IsAfter + (1 | ProjectLanguage) +  
    (1 | ProjectName)
   Data: NumOfReviewCommentsData

      AIC       BIC    logLik  deviance  df.resid 
 648004.4  648167.0 -323986.2  647972.4    190478 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.5935 -0.6894 -0.1692  0.3620  4.0995 

Random effects:
 Groups          Name        Variance Std.Dev.
 ProjectName     (Intercept) 0.122089 0.34941 
 ProjectLanguage (Intercept) 0.001171 0.03422 
 Residual                    1.751272 1.32336 
Number of obs: 190494, groups:  ProjectName, 154; ProjectLanguage, 31

Fixed effects:
              

R2m       R2c 
0.2598575 0.3085257

log(Additions + 1)        log(Deletions + 1)              IsMergedTrue 
                 3.166399                  2.625062                  1.087998 
         log(Commits + 1)        log(Assignees + 1)     log(ChangedFiles + 1) 
                 1.704232                  1.015183                  3.458589 
log(NumOfUniqueUsers + 1)        log(PRsOpened + 1)      log(TotalBuilds + 1) 
                 1.116322                  2.310020                  2.710156 
       NewContributorTrue          FromOutsiderTrue               IsAfterTrue 
                 1.494011                  1.587954                  2.599235

,Sum Sq,Mean Sq,NumDF,DenDF,F.value,Pr(>F)
log(Additions + 1),5243.725642,5243.725642,1,190479.7,2.994238e+03,0.000000e+00
log(Deletions + 1),33.676712,33.676712,1,190437.6,1.922985e+01,1.159489e-05
IsMerged,3214.311170,3214.311170,1,190489.0,1.835415e+03,0.000000e+00
log(Commits + 1),2922.682116,2922.682116,1,190486.5,1.668891e+03,0.000000e+00
log(Assignees + 1),482.935633,482.935633,1,184915.9,2.757627e+02,0.000000e+00
log(ChangedFiles + 1),1397.113229,1397.113229,1,190473.9,7.977704e+02,0.000000e+00
log(NumOfUniqueUsers + 1),65114.667106,65114.667106,1,189997.5,3.718135e+04,0.000000e+00
log(PRsOpened + 1),1.392224,1.392224,1,189790.2,7.949787e-01,3.726000e-01
log(TotalBuilds + 1),383.466668,383.466668,1,190471.4,2.189646e+02,0.000000e+00
NewContributor,198.736678,198.736678,1,190491.3,1.134813e+02,0.000000e+00


[1] "---------------------------"


Linear mixed model fit by maximum likelihood t-tests use Satterthwaite
  approximations to degrees of freedom [lmerMod]
Formula: log(ShareReviewComments + 1) ~ log(Additions + 1) + log(Deletions +  
    1) + log(Commits + 1) + log(Assignees + 1) + log(ChangedFiles +  
    1) + log(NumOfUniqueUsers + 1) + log(PRsOpened + 1) + log(TotalBuilds +  
    1) + NewContributor + IsAfter + (1 | ProjectLanguage) + (1 |  
    ProjectName)
   Data: hasReviewCommentsAndFromOutsider

     AIC      BIC   logLik deviance df.resid 
 36306.2  36417.4 -18139.1  36278.2    20822 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-5.2019 -0.5980  0.1838  0.7364  2.6790 

Random effects:
 Groups          Name        Variance  Std.Dev. 
 ProjectName     (Intercept) 3.310e-02 1.819e-01
 ProjectLanguage (Intercept) 3.621e-14 1.903e-07
 Residual                    3.287e-01 5.733e-01
Number of obs: 20836, groups:  ProjectName, 154; ProjectLanguage, 31

Fixed effects:
                            Estima

R2m       R2c 
0.1382025 0.2170456

log(Additions + 1)        log(Deletions + 1)          log(Commits + 1) 
                 3.180583                  2.266995                  1.585993 
       log(Assignees + 1)     log(ChangedFiles + 1) log(NumOfUniqueUsers + 1) 
                 1.011623                  3.503254                  1.072320 
       log(PRsOpened + 1)      log(TotalBuilds + 1)        NewContributorTrue 
                 1.656486                  2.939149                  1.614162 
              IsAfterTrue 
                 2.692700

,Sum Sq,Mean Sq,NumDF,DenDF,F.value,Pr(>F)
log(Additions + 1),32.5051802,32.5051802,1,20835.71,98.8814315,0.000000e+00
log(Deletions + 1),0.0785066,0.0785066,1,20812.89,0.2388187,6.250658e-01
log(Commits + 1),6.0807192,6.0807192,1,20827.73,18.4976739,1.708859e-05
log(Assignees + 1),1.6567522,1.6567522,1,19604.97,5.0398745,2.478139e-02
log(ChangedFiles + 1),7.1808298,7.1808298,1,20834.90,21.8442329,2.975540e-06
log(NumOfUniqueUsers + 1),835.2846935,835.2846935,1,20544.46,2540.9533376,0.000000e+00
log(PRsOpened + 1),10.1896490,10.1896490,1,20706.70,30.9971232,2.616164e-08
log(TotalBuilds + 1),0.1891865,0.1891865,1,20822.34,0.5755092,4.480866e-01
NewContributor,0.6773032,0.6773032,1,20813.06,2.0603704,1.511884e-01
IsAfter,1.7540735,1.7540735,1,20827.32,5.3359280,2.089994e-02


modelBuildDiscussionComments = lmer(log(GeneralCommentsDiscussingBuild + 1) ~ 
            log(Additions + 1) +
            log(Deletions + 1) +
            IsMerged +
            log(Commits + 1) +
            log(Assignees + 1) + 
            log(ChangedFiles + 1) + 
            log(PrOpenedDaysAfterProjectStart + 1) +
            IsAfter + 
            (1|ProjectLanguage) +
            (1|ProjectName),
          data= hasGeneralComments, 
          REML=FALSE)
summary(modelBuildDiscussionComments)
r.squaredGLMM(modelBuildDiscussionComments)
vif.mer(modelBuildDiscussionComments)
anova(modelBuildDiscussionComments)

In [147]:
library(lme4)

print(sprintf("R2c of review comments is %f", r.squaredGLMM(modelNumberReviewComments)[['R2c']]))
print(sprintf("R2c of share review comments is %f", r.squaredGLMM(modelShareReviewComments)[['R2c']]))
print(sprintf("R2c of general comments is %f", r.squaredGLMM(modelNumberGeneralComments)[['R2c']]))

[1] "R2c of review comments is 0.336579"
[1] "R2c of share review comments is 0.308694"
[1] "R2c of general comments is 0.717783"
